# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
data = pd.read_csv('event_datafile_new.csv')
data

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Sydney Youngblood,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",954,Ain't No Sunshine,73
1,Gang Starr,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",984,My Advice 2 You (Explicit),24
2,3OH!3,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",984,My First Kiss (Feat. Ke$ha) [Album Version],24
3,RÃÂ¶yksopp,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",954,The Girl and The Robot,73
4,Kajagoogoo,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",984,Too Shy,24
5,The Jimmy Castor Bunch,Layla,F,91,Griffin,306.54649,paid,"Lake Havasu City-Kingman, AZ",984,Potential,24
6,Ministry,Jacob,M,55,Klein,186.69669,paid,"Tampa-St. Petersburg-Clearwater, FL",954,The Great Satan (What Would Satan Do Mix),73
7,Bouncing Souls,Jacob,M,56,Klein,117.31546,paid,"Tampa-St. Petersburg-Clearwater, FL",954,The Screamer,73
8,Tegan And Sara,Layla,F,92,Griffin,129.67138,paid,"Lake Havasu City-Kingman, AZ",984,Knife Going In (Album Version),24
9,Silverchair,Layla,F,93,Griffin,242.96444,paid,"Lake Havasu City-Kingman, AZ",984,The Greatest View (Album Version),24


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [14]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [15]:
# TO-DO: Create a Keyspace 
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS udacity_project WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")
except cassandra.cluster.NoHostAvailable as e:
    print(e)

#### Set Keyspace

In [16]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_project')
except cassandra.cluster.NoHostAvailable as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [24]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4

## I think the sessionId and itemInSession are unique enough for the purpose of this query.
##Query: SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4;
query = "CREATE TABLE IF NOT EXISTS music_library (artist text, firstName text, gender text, itemInSession int, "
query = query + "lastName text, length float, level text, location text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
    print("Table created successfully")
except cassandra.cluster.NoHostAvailable as e:
    print(e)

Table created successfully


In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

# Just to indicate the the data insertion operation is completed
print("Completed")        

Completed


#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = "SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [33]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# We will use userId and sessionId as the primary key and itemInSession, FirstName and LastName as clustering column.
# The query only specify ItemInSession as the clustering column.
query = "CREATE TABLE IF NOT EXISTS artist_library (artist text, firstName text, gender text, itemInSession int, "
query = query + "lastName text, length float, level text, location text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
    print("Table created successfully")
except cassandra.cluster.NoHostAvailable as e:
    print(e)

                    

Table created successfully


In [34]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        
print("Completed") 

Completed


In [43]:
query = "SELECT artist, song, firstName, lastName FROM artist_library WHERE userId = 10 AND sessionId=182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, "|", row.song, "|", row.firstname, "|", row.lastname)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                    

In [45]:
query = "CREATE TABLE IF NOT EXISTS song_library (artist text, firstName text, gender text, itemInSession int, "
query = query + "lastName text, length float, level text, location text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY (song, userId, sessionId))"
try:
    session.execute(query)
    print("Table created successfully")
except cassandra.cluster.NoHostAvailable as e:
    print(e)


Table created successfully


In [46]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        
print("Completed") 

Completed


In [47]:
query = "SELECT firstName, lastName FROM song_library WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, "|", row.lastname)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [48]:
query = "drop table music_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [49]:
query = "drop table artist_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [50]:
query = "drop table song_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [51]:
session.shutdown()
cluster.shutdown()